In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
IMG_PATH="../facial_expressions/"

In [4]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [5]:
face_dim=128

In [6]:
import matplotlib.pyplot as plt

In [7]:
def ret_keypoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)==0:
        param=[0,0,face_dim,face_dim]
    else:
        (x,y,w,h)=faces[0]
        param=[x,y,x+w,y+h]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [8]:
def prep_keypoints():
    X_inp=[]
    y_inp=[]
    for em in emotions:
        ltt=os.listdir(IMG_PATH+em)
        lnltt=len(ltt)
        for idx,imn in enumerate(ltt):
            print("\r",idx+1,'/',lnltt,end=" ")
            image = cv2.imread(IMG_PATH+em+"/"+imn)
            landmarks=ret_keypoints(image)
            if landmarks is not None:
                X_inp.append(landmarks)
                yy=np.zeros(len(emotions))
                yy[emotions.index(em)]=1
                y_inp.append(yy)
        print()
    return np.asarray(X_inp),np.asarray(y_inp)

In [10]:
XXD,YYD=prep_keypoints()

 124 / 124 
 84 / 84 
 67 / 67 
 127 / 127 
 132 / 132 

 123 / 123 


In [11]:
s=np.arange(len(XXD))
for i in range(5):
    np.random.shuffle(s)
    XXD=XXD[s]
    YYD=YYD[s]

In [12]:
import pickle

In [13]:
with open("extra_data.pkl","wb") as f:
    pickle.dump((XXD,YYD), f)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [12]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(emotions),activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              140288    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
cut=80
XDB=XXD[cut:]
YDB=YYD[cut:]

XTB=XXD[:cut]
YTB=YYD[:cut]

In [22]:
model.fit(XDB,YDB,batch_size=32,epochs=100,validation_data=(XTB,YTB))

Train on 577 samples, validate on 80 samples
Epoch 1/100
577/577 [==============================] - 0s 349us/sample - loss: 0.6789 - accuracy: 0.7331 - val_loss: 4.2401 - val_accuracy: 0.2125
Epoch 2/100
577/577 [==============================] - 0s 368us/sample - loss: 0.6672 - accuracy: 0.7348 - val_loss: 3.2057 - val_accuracy: 0.3125
Epoch 3/100
577/577 [==============================] - 0s 379us/sample - loss: 0.6158 - accuracy: 0.7331 - val_loss: 4.0774 - val_accuracy: 0.3375
Epoch 4/100
577/577 [==============================] - 0s 427us/sample - loss: 0.6851 - accuracy: 0.7383 - val_loss: 1.2222 - val_accuracy: 0.6000
Epoch 5/100
577/577 [==============================] - 0s 426us/sample - loss: 0.6544 - accuracy: 0.7591 - val_loss: 1.3314 - val_accuracy: 0.5250
Epoch 6/100
577/577 [==============================] - 0s 521us/sample - loss: 0.5997 - accuracy: 0.7730 - val_loss: 1.6246 - val_accuracy: 0.5625
Epoch 7/100
577/577 [==============================] - 0s 393us/sample - 

In [17]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.325

In [18]:
model.predict(XTB).argmax(axis=1)

array([3, 3, 3, 3, 5, 2, 5, 3, 2, 5, 2, 0, 2, 2, 5, 3, 2, 3, 3, 5, 2, 2,
       2, 5, 3, 3, 3, 5, 3, 2, 5, 5, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 0, 3,
       2, 3, 2, 2, 5, 4, 3, 2, 2, 2, 3, 2, 5, 2, 2, 3, 5, 3, 2, 2, 3, 2,
       3, 2, 2, 2, 0, 5, 3, 3, 2, 3, 2, 5, 5, 3])

In [19]:
YTB.argmax(axis=1)

array([3, 3, 3, 0, 5, 4, 5, 2, 5, 5, 4, 1, 0, 5, 5, 3, 5, 3, 0, 0, 0, 0,
       4, 0, 3, 3, 0, 2, 3, 5, 3, 0, 0, 5, 1, 4, 2, 3, 0, 1, 3, 1, 0, 2,
       2, 4, 0, 5, 0, 0, 4, 2, 4, 1, 3, 5, 5, 0, 4, 1, 5, 3, 3, 5, 0, 4,
       0, 0, 2, 4, 0, 5, 1, 1, 4, 4, 2, 0, 5, 0])

In [23]:
cam = cv2.VideoCapture(0)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    landmarks=[]
    for (x,y,w,h) in faces:
        svx,svy=x,y
        gray=gray[y:y+h,x:x+w]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            print(gray,x,y,w,h)
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 2, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
    if len(landmarks)>0:
        y_out=model.predict(np.asarray(landmarks))
        res=np.argmax(y_out,axis=1)
        for r in res:
            cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

[] 231 201 186 186
[] 230 201 187 187
[] 233 201 185 185
[] 231 201 186 186
[] 230 200 188 188
[] 224 196 196 196
[] 225 197 190 190
[] 223 196 196 196
[] 222 196 196 196
[] 224 195 196 196
[] 222 196 196 196
[] 222 196 196 196
[] 225 198 192 192
[] 224 196 196 196
[] 224 196 196 196
[] 223 196 196 196
[] 82 202 116 116
[] 80 203 116 116
[] 228 201 192 192
[] 226 199 196 196
[] 225 197 196 196
[] 224 196 196 196
[] 224 196 196 196
[] 224 195 196 196
[] 225 195 196 196
[] 222 196 196 196
[] 224 196 196 196
[] 225 197 196 196
[] 225 196 196 196
[] 226 196 196 196
[] 222 197 196 196
[] 229 199 192 192
[] 226 199 196 196
[] 226 201 192 192
[] 227 201 189 189
[] 223 199 196 196
[] 224 201 191 191
[] 219 199 196 196
[] 68 203 112 112
[] 220 200 196 196
[] 222 200 196 196
[] 222 200 196 196
[] 224 203 196 196
[] 225 202 192 192
[] 66 202 112 112
[] 221 203 196 196
[] 223 202 196 196
[] 225 203 192 192
[] 220 200 196 196
[] 222 200 196 196
[] 222 199 201 201
[] 222 200 196 196
[] 222 200 196 1

In [21]:
cam.release()
cv2.destroyAllWindows()